In [1]:
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
service = QiskitRuntimeService()


In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram

def bit_compare():
    qr = QuantumRegister(2, "bits")
    aux = QuantumRegister(2, "aux")
    
    qc = QuantumCircuit(qr, aux)
    qc.x(qr[1])
    qc.mcx(qr, aux[0])
    qc.x(qr[0])
    qc.x(qr[1])
    qc.mcx(qr, aux[1])
    qc.x(qr[0])
    
    return qc

In [3]:
print(bit_compare())

                  ┌───┐     ┌───┐
bits_0: ───────■──┤ X ├──■──┤ X ├
        ┌───┐  │  ├───┤  │  └───┘
bits_1: ┤ X ├──■──┤ X ├──■───────
        └───┘┌─┴─┐└───┘  │       
 aux_0: ─────┤ X ├───────┼───────
             └───┘     ┌─┴─┐     
 aux_1: ───────────────┤ X ├─────
                       └───┘     


In [4]:
def encode(bit):
    qr = QuantumRegister(1, "number")
    qc = QuantumCircuit(qr)
    if (bit == "1"):
        qc.x(qr[0])
    return qc

In [5]:
def compare_bitstring(bitstring_a, bitstring_b):
    bits = len(bitstring_a)
    qra = QuantumRegister(bits, "a")
    qrb = QuantumRegister(bits, "b")
    qraux = QuantumRegister(2*bits, "aux")
    qrint = QuantumRegister(bits-1, "int")
    cr = ClassicalRegister(4)

    qc = QuantumCircuit(qra, qrb, qraux, qrint, cr)

    for i in range(bits):
        qc.append(encode(bitstring_a[i]), [qra[i]])
        qc.append(encode(bitstring_b[i]), [qrb[i]])
        qc.append(bit_compare(), [qra[i], qrb[i], qraux[2*i], qraux[2*i+1]])
                
        if i < bits-1:
            qc.x(qraux[2*i])
            qc.x(qraux[2*i+1])
            qc.mcx([qraux[2*i], qraux[2*i+1]], qrint[i])
            qc.x(qraux[2*i])
            qc.x(qraux[2*i+1])
        
    for i in range(0, bits-1):
        qc.mcx([qraux[2*(-i-1)],  qrint[-i]], qraux[2*(-i)])
        qc.mcx([qraux[2*(-i-1)+1],  qrint[-i]], qraux[2*(-i)+1])
        
    qc.measure(qraux[0], cr[0])
    qc.measure(qraux[1], cr[1])
       
       
    return qc

In [6]:
#circuit = compare_bitstring("11", "10")

In [7]:
circuit = compare_bitstring("110", "111")

In [8]:
print(circuit)

       ┌─────────────┐┌──────────────┐                                     »
  a_0: ┤ circuit-160 ├┤0             ├─────────────────────────────────────»
       ├─────────────┤│              │┌──────────────┐                     »
  a_1: ┤ circuit-169 ├┤              ├┤0             ├─────────────────────»
       ├─────────────┤│              ││              │┌──────────────┐     »
  a_2: ┤ circuit-178 ├┤              ├┤              ├┤0             ├─────»
       ├─────────────┤│              ││              ││              │     »
  b_0: ┤ circuit-163 ├┤1             ├┤              ├┤              ├─────»
       ├─────────────┤│  circuit-166 ││              ││              │     »
  b_1: ┤ circuit-172 ├┤              ├┤1             ├┤              ├─────»
       ├─────────────┤│              ││              ││              │     »
  b_2: ┤ circuit-181 ├┤              ├┤  circuit-175 ├┤1             ├─────»
       └─────────────┘│              ││              ││              │┌───┐»

In [9]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import transpile
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
service = QiskitRuntimeService()
#backend = service.backend("ibmq_qasm_simulator")
backend = service.get_backend("simulator_statevector")
pm = generate_preset_pass_manager(optimization_level=1, target=backend.target) 
isa_circuits = pm.run(circuit)
job = Sampler(backend).run(isa_circuits)
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

/home/bram/.venvs/qiskit312/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:927: UserWarning: Cloud simulators will be deprecated on 15 May 2024. Use the new local testing mode in qiskit-ibm-runtime version 0.22.0 or later to meet your debugging needs.
  warnings.warn(warning_message)


job id: co7f7costs3phnu5cmng
SamplerResult(quasi_dists=[{2: 1.0}], metadata=[{'shots': 4000, 'circuit_metadata': {}}])
